<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-12 13:11:21
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.35 C
-------------------
Today PnL: -35.73 K (-0.26%)
Current PnL: -14.52 L (-10.12%)
CY Booked + Current PnL: -8.15 L (-5.68%)
-------------------
Total profit:  3.52 L
Total loss:  -18.05 L
-------------------
Total Booked + Current PnL: 18.83 L (16.15%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.7%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 70.41 L (51.99%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-3.65,H-LC,86.81,98231.0,13044.0,3016.0,-1.15,15.31,3.07,18.85,35.0,4.32,0.74,30.79,XY25,NTT,INSURANCE
60,RELIANCE,1291.83,1526.0,6.50,X-LC,28.57,158006.0,17197.0,8327.0,0.05,12.21,5.27,18.13,17.0,2.07,1.20,24.76,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,3.24,M-LC,53.85,103603.0,25241.0,8786.0,0.19,32.21,8.48,43.43,77.0,2.87,0.79,38.57,XY24,NTT,MISC
82,UJJIVANSFB,52.77,53.0,54.03,M-SC,83.52,133947.0,-8532.0,9149.0,1.22,-5.99,6.83,0.44,242.0,-0.93,1.02,58.09,OX40N,NTT,BANKS
76,TATAELXSI,7332.28,7332.0,-12.79,X-MC,57.14,91637.0,-11015.0,11015.0,-1.19,-10.73,12.02,-0.00,57.0,-1.00,0.69,39.30,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,561.33,680.00,-16.51,H-MC,45.05,228825.0,1486.0,46566.0,-0.39,0.65,20.35,21.14,117.0,0.03,1.73,28.12,XY24,NTT,PAINTS
54,NATIONALUM,189.63,247.44,-36.43,H-SC,71.43,100334.0,-170.0,30813.0,-0.37,-0.17,30.71,30.49,97.0,-0.01,0.76,32.18,X5K,ATH,METALS
56,PGHH,13388.00,18062.58,-20.14,X-MC,7.69,201480.0,660.0,69450.0,0.13,0.33,34.47,34.92,53.0,0.01,1.53,5.02,X40,ATH,FMCG
30,HAVELLS,1588.02,2069.16,-4.90,X-LC,18.68,245778.0,-1953.0,77002.0,-0.23,-0.79,31.33,30.30,52.0,-0.03,1.86,12.40,X40,ATH,ELECTRICAL
33,HINDALCO,651.95,761.55,-12.54,H-LC,25.27,104848.0,536.0,16996.0,0.02,0.51,16.21,16.81,26.0,0.03,0.79,16.64,X5K,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TANLA,917.30,1963.11,-27.37,H-SC,98.90,177872.0,-61543.0,334506.0,9.45,-25.71,188.06,114.01,128.0,-0.18,1.35,63.14,AR,ATH,IT
62,ROUTE,1558.40,2249.26,-33.00,M-SC,84.62,72181.0,-30673.0,76274.0,6.49,-29.82,105.67,44.33,188.0,-0.40,0.55,19.87,SR,ATH,IT
85,VALIANTORG,512.64,838.00,-402.56,H-SC,100.00,128648.0,-4638.0,89230.0,2.31,-3.48,69.36,63.47,142.0,-0.05,0.98,115.65,XR,NTT,CHEMICALS
42,INFY,1461.46,2275.00,-8.83,X-LC,31.87,291967.0,30366.0,115269.0,2.20,11.61,39.48,55.67,1.0,0.26,2.21,18.42,X40,BTT,IT
51,LTIM,5564.16,7197.33,2.73,H-LC,72.53,197874.0,-2436.0,61222.0,1.90,-1.22,30.94,29.35,36.0,-0.04,1.50,36.70,X200,ATH,IT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69677.0,-43872.0,83926.0,-2.79,-38.64,120.45,35.27,260.0,-0.52,0.53,70.54,XR,NTT,HOTELS
27,GREENPANEL,375.16,537.00,199.74,M-SC,87.91,133810.0,-49268.0,128244.0,-2.47,-26.91,95.84,43.14,212.0,-0.38,1.01,25.87,XY24,NTT,MISC
23,DABUR,505.20,735.00,-14.56,X-MC,19.78,185250.0,-11778.0,101406.0,-2.36,-5.98,54.74,45.49,54.0,-0.12,1.40,5.23,XY24,BTT,FMCG
17,CAMS,3643.00,5250.99,2.89,H-SC,73.63,115699.0,13695.0,31331.0,-2.29,13.43,27.08,44.14,125.0,0.44,0.88,31.85,X40N,ATH,MISC
9,AWL,325.39,485.00,-68.39,H-MC,40.66,222709.0,-45738.0,177410.0,-2.19,-17.04,79.66,49.05,98.0,-0.26,1.69,14.51,XY24,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,54.03,M-SC,83.52,133947.0,-8532.0,9149.0,1.22,-5.99,6.83,0.44,242.0,-0.93,1.02,58.09,OX40N,NTT,BANKS
74,SYMPHONY,1306.42,1306.0,-27.66,M-SC,3.30,154043.0,-17098.0,17037.0,-1.17,-9.99,11.06,-0.03,190.0,-1.00,1.17,10.71,OX40N,NTT,DURABLES
76,TATAELXSI,7332.28,7332.0,-12.79,X-MC,57.14,91637.0,-11015.0,11015.0,-1.19,-10.73,12.02,-0.00,57.0,-1.00,0.69,39.30,OX40N,NTT,IT
87,VIPIND,488.80,489.0,-952.17,H-SC,93.41,81218.0,-14098.0,14140.0,0.70,-14.79,17.41,0.04,153.0,-1.00,0.62,60.19,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.49,H-SC,9.89,225945.0,-43722.0,80052.0,0.10,-16.21,35.43,13.47,144.0,-0.55,1.71,13.09,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-8.83,X-LC,31.87,291967.0,30366.0,115269.0,2.20,11.61,39.48,55.67,1.0,0.26,2.21,18.42,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.12,X-LC,8.79,269732.0,-26202.0,120112.0,-0.40,-8.85,44.53,31.73,2.0,-0.22,2.04,7.96,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-16.46,X-LC,14.29,262450.0,-7897.0,42517.0,-1.39,-2.92,16.20,12.81,5.0,-0.19,1.99,8.62,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,12.87,X-LC,24.18,258911.0,29678.0,37620.0,-0.38,12.95,14.53,29.35,7.0,0.79,1.96,23.02,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-11.96,X-LC,23.08,279747.0,14321.0,42577.0,-1.17,5.40,15.22,21.44,10.0,0.34,2.12,11.93,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UNIONBANK,123.87,163.0,-0.31,M-LC,89.01,173483.0,32643.0,11849.0,-0.16,23.18,6.83,31.59,86.0,2.75,1.31,50.70,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,3.24,M-LC,53.85,103603.0,25241.0,8786.0,0.19,32.21,8.48,43.43,77.0,2.87,0.79,38.57,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,130.64,228.00,100.16,M-MC,70.33,227821.0,2859.0,164806.0,-0.64,1.27,72.34,74.53,184.0,0.02,1.73,32.27,XY24,BTT,STEEL
3,ADANIPORTS,1103.69,1583.00,3.24,M-LC,53.85,103603.0,25241.0,8786.0,0.19,32.21,8.48,43.43,77.0,2.87,0.79,38.57,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,2.89,H-SC,73.63,115699.0,13695.0,31331.0,-2.29,13.43,27.08,44.14,125.0,0.44,0.88,31.85,X40N,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,63.74,154038.0,3125.0,57333.0,-0.21,2.07,37.22,40.06,100.0,0.05,1.17,25.99,AR,ATH,AUTO
83,UNIONBANK,123.87,163.00,-0.31,M-LC,89.01,173483.0,32643.0,11849.0,-0.16,23.18,6.83,31.59,86.0,2.75,1.31,50.70,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-31.38,M-SC,52.75,199964.0,28343.0,133716.0,0.80,16.51,66.87,94.43,194.0,0.21,1.52,47.59,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-5.17,M-LC,37.36,162316.0,11371.0,98088.0,1.11,7.53,60.43,72.51,68.0,0.12,1.23,11.98,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-28.22,M-MC,76.92,191127.0,11319.0,101087.0,0.26,6.30,52.89,62.52,170.0,0.11,1.45,37.54,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.23,H-SC,60.44,132394.0,9058.0,125774.0,-0.00,7.34,95.00,109.32,122.0,0.07,1.00,30.80,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,63.74,154038.0,3125.0,57333.0,-0.21,2.07,37.22,40.06,100.0,0.05,1.17,25.99,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69677.0,-43872.0,83926.0,-2.79,-38.64,120.45,35.27,260.0,-0.52,0.53,70.54,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-21.41,H-SC,65.93,89385.0,-8595.0,30462.0,-1.14,-8.77,34.08,22.32,141.0,-0.28,0.68,29.64,XR,ATH,FINANCE
50,LICI,827.06,983.00,-3.65,H-LC,86.81,98231.0,13044.0,3016.0,-1.15,15.31,3.07,18.85,35.0,4.32,0.74,30.79,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,3.24,M-LC,53.85,103603.0,25241.0,8786.0,0.19,32.21,8.48,43.43,77.0,2.87,0.79,38.57,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,39.23,H-LC,90.11,104310.0,1350.0,44634.0,0.19,1.31,42.79,44.67,25.0,0.03,0.79,34.61,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VALIANTORG,512.64,838.00,-402.56,H-SC,100.0,128648.0,-4638.0,89230.0,2.31,-3.48,69.36,63.47,142.0,-0.05,0.98,115.65,XR,NTT,CHEMICALS
75,TANLA,917.30,1963.11,-27.37,H-SC,98.9,177872.0,-61543.0,334506.0,9.45,-25.71,188.06,114.01,128.0,-0.18,1.35,63.14,AR,ATH,IT
7,ASIANTILES,75.41,137.00,6816.67,M-SC,97.8,75909.0,-17901.0,94522.0,-1.79,-19.08,124.52,81.67,238.0,-0.19,0.58,47.00,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-32.18,L-SC,96.7,137980.0,-3862.0,60435.0,-0.00,-2.72,43.80,39.88,258.0,-0.06,1.05,39.83,XY24,NTT,POWER
79,TITAGARH,1097.23,1548.00,-7.75,H-SC,95.6,162225.0,-29790.0,108675.0,-1.00,-15.51,66.99,41.08,151.0,-0.27,1.23,37.30,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,43.14
2,50,72.00


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.05
XY25     14.26
X40      12.50
XR       11.06
OX40N     8.53
X40N      8.44
AR        8.11
X5K       2.34
X200      1.50
SR        1.20
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.86
X-LC    15.50
M-SC    14.41
H-LC    11.04
H-MC     8.95
M-LC     8.55
X-MC     7.68
M-MC     5.82
X-SC     2.80
L-SC     2.52
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.56,-1.89,35.96
IT,11.41,-10.19,66.13
BANKS,8.61,-7.23,50.44
MISC,6.61,-24.02,72.51
FINANCE,6.44,-18.42,63.98
PAINTS,5.64,-17.15,40.23
ELECTRICAL,5.15,-3.25,48.03
HEALTHCARE,4.76,-2.63,32.61
AUTO,4.71,-8.87,56.74


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2703144.0,25
XR,1102228.0,13
AR,999842.0,9
X40,668308.0,9
XY25,507475.0,10
X40N,414774.0,8
OX40N,305375.0,11
SR,186478.0,2
X5K,92443.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1877378.0,19
M-SC,1429385.0,18
X-LC,616924.0,9
M-MC,576038.0,5
H-MC,545507.0,7
H-LC,496803.0,10
X-MC,403184.0,6
M-LC,317117.0,6
L-SC,265626.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,886707.0,6
M-SC,XY24,571988.0,5
H-SC,AR,514301.0,3
M-MC,XY24,388677.0,3
M-SC,XR,342389.0,4
X-LC,X40,312383.0,3
H-MC,XY24,223976.0,2
M-LC,XY24,205091.0,4
H-SC,XR,191705.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
